# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f95c09a2ee0>
├── 'a' --> tensor([[ 1.5831,  0.1584, -0.5722],
│                   [-1.3059,  0.2935, -0.6806]])
└── 'x' --> <FastTreeValue 0x7f95c09a2eb0>
    └── 'c' --> tensor([[ 1.4959, -0.3282,  0.1670,  0.1570],
                        [ 0.5016,  0.3339,  0.7168,  0.5720],
                        [ 0.8196, -0.8302,  0.5668, -1.7408]])

In [4]:
t.a

tensor([[ 1.5831,  0.1584, -0.5722],
        [-1.3059,  0.2935, -0.6806]])

In [5]:
%timeit t.a

61.9 ns ± 0.0407 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f95c09a2ee0>
├── 'a' --> tensor([[-0.1568, -0.8128, -0.0317],
│                   [-0.3456,  1.1366,  1.2238]])
└── 'x' --> <FastTreeValue 0x7f95c09a2eb0>
    └── 'c' --> tensor([[ 1.4959, -0.3282,  0.1670,  0.1570],
                        [ 0.5016,  0.3339,  0.7168,  0.5720],
                        [ 0.8196, -0.8302,  0.5668, -1.7408]])

In [7]:
%timeit t.a = new_value

59.2 ns ± 0.0145 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.5831,  0.1584, -0.5722],
               [-1.3059,  0.2935, -0.6806]]),
    x: Batch(
           c: tensor([[ 1.4959, -0.3282,  0.1670,  0.1570],
                      [ 0.5016,  0.3339,  0.7168,  0.5720],
                      [ 0.8196, -0.8302,  0.5668, -1.7408]]),
       ),
)

In [10]:
b.a

tensor([[ 1.5831,  0.1584, -0.5722],
        [-1.3059,  0.2935, -0.6806]])

In [11]:
%timeit b.a

52.8 ns ± 0.0754 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.1407,  0.9980,  0.0763],
               [-0.7677, -0.7290, -0.6322]]),
    x: Batch(
           c: tensor([[ 1.4959, -0.3282,  0.1670,  0.1570],
                      [ 0.5016,  0.3339,  0.7168,  0.5720],
                      [ 0.8196, -0.8302,  0.5668, -1.7408]]),
       ),
)

In [13]:
%timeit b.a = new_value

478 ns ± 0.138 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

832 ns ± 0.188 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 40.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 276 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 303 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f95c09650d0>
├── 'a' --> tensor([[[ 1.5831,  0.1584, -0.5722],
│                    [-1.3059,  0.2935, -0.6806]],
│           
│                   [[ 1.5831,  0.1584, -0.5722],
│                    [-1.3059,  0.2935, -0.6806]],
│           
│                   [[ 1.5831,  0.1584, -0.5722],
│                    [-1.3059,  0.2935, -0.6806]],
│           
│                   [[ 1.5831,  0.1584, -0.5722],
│                    [-1.3059,  0.2935, -0.6806]],
│           
│                   [[ 1.5831,  0.1584, -0.5722],
│                    [-1.3059,  0.2935, -0.6806]],
│           
│                   [[ 1.5831,  0.1584, -0.5722],
│                    [-1.3059,  0.2935, -0.6806]],
│           
│                   [[ 1.5831,  0.1584, -0.5722],
│                    [-1.3059,  0.2935, -0.6806]],
│           
│                   [[ 1.5831,  0.1584, -0.5722],
│                    [-1.3059,  0.2935, -0.6806]]])
└── 'x' --> <FastTreeValue 0x7f95c09656a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 46.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f95c09fbdf0>
├── 'a' --> tensor([[ 1.5831,  0.1584, -0.5722],
│                   [-1.3059,  0.2935, -0.6806],
│                   [ 1.5831,  0.1584, -0.5722],
│                   [-1.3059,  0.2935, -0.6806],
│                   [ 1.5831,  0.1584, -0.5722],
│                   [-1.3059,  0.2935, -0.6806],
│                   [ 1.5831,  0.1584, -0.5722],
│                   [-1.3059,  0.2935, -0.6806],
│                   [ 1.5831,  0.1584, -0.5722],
│                   [-1.3059,  0.2935, -0.6806],
│                   [ 1.5831,  0.1584, -0.5722],
│                   [-1.3059,  0.2935, -0.6806],
│                   [ 1.5831,  0.1584, -0.5722],
│                   [-1.3059,  0.2935, -0.6806],
│                   [ 1.5831,  0.1584, -0.5722],
│                   [-1.3059,  0.2935, -0.6806]])
└── 'x' --> <FastTreeValue 0x7f9508c950a0>
    └── 'c' --> tensor([[ 1.4959, -0.3282,  0.1670,  0.1570],
                        [ 0.5016,  0.3339,  0.7168,  0.5720],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 27.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.6 µs ± 50.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.5831,  0.1584, -0.5722],
                [-1.3059,  0.2935, -0.6806]],
       
               [[ 1.5831,  0.1584, -0.5722],
                [-1.3059,  0.2935, -0.6806]],
       
               [[ 1.5831,  0.1584, -0.5722],
                [-1.3059,  0.2935, -0.6806]],
       
               [[ 1.5831,  0.1584, -0.5722],
                [-1.3059,  0.2935, -0.6806]],
       
               [[ 1.5831,  0.1584, -0.5722],
                [-1.3059,  0.2935, -0.6806]],
       
               [[ 1.5831,  0.1584, -0.5722],
                [-1.3059,  0.2935, -0.6806]],
       
               [[ 1.5831,  0.1584, -0.5722],
                [-1.3059,  0.2935, -0.6806]],
       
               [[ 1.5831,  0.1584, -0.5722],
                [-1.3059,  0.2935, -0.6806]]]),
    x: Batch(
           c: tensor([[[ 1.4959, -0.3282,  0.1670,  0.1570],
                       [ 0.5016,  0.3339,  0.7168,  0.5720],
                       [ 0.8196, -0.8302,  0.5668, -1.7408]],
         

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 170 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.5831,  0.1584, -0.5722],
               [-1.3059,  0.2935, -0.6806],
               [ 1.5831,  0.1584, -0.5722],
               [-1.3059,  0.2935, -0.6806],
               [ 1.5831,  0.1584, -0.5722],
               [-1.3059,  0.2935, -0.6806],
               [ 1.5831,  0.1584, -0.5722],
               [-1.3059,  0.2935, -0.6806],
               [ 1.5831,  0.1584, -0.5722],
               [-1.3059,  0.2935, -0.6806],
               [ 1.5831,  0.1584, -0.5722],
               [-1.3059,  0.2935, -0.6806],
               [ 1.5831,  0.1584, -0.5722],
               [-1.3059,  0.2935, -0.6806],
               [ 1.5831,  0.1584, -0.5722],
               [-1.3059,  0.2935, -0.6806]]),
    x: Batch(
           c: tensor([[ 1.4959, -0.3282,  0.1670,  0.1570],
                      [ 0.5016,  0.3339,  0.7168,  0.5720],
                      [ 0.8196, -0.8302,  0.5668, -1.7408],
                      [ 1.4959, -0.3282,  0.1670,  0.1570],
                      [ 0.5016,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 209 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

323 µs ± 14 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
